In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
from pprint import pprint

from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import torch 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [4]:
ROOT_DIR = "data"
RANDOM_STATE = 42

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

# 모든 값이 동일할 때 컬럼을 제거
train_data = train_data.loc[:, (train_data != train_data.iloc[0]).any()]
test_data = test_data.loc[:, (test_data != test_data.iloc[0]).any()]

# 모든 값이 NAN인 열 제거
# train_data = train_data.dropna(axis=1, how='all')
# test_data = test_data.dropna(axis=1, how='all')
train_data.dropna(axis=1, inplace=True)
test_data.dropna(axis=1, inplace=True)

def add_vector_magnitude(df, patterns):
    for pattern, new_column_name in patterns.items():
        columns_to_drop = []
        
        # 지정된 패턴에 맞는 컬럼 선택
        selected_columns = df.filter(regex=pattern) 
        
        if not selected_columns.empty:

            # 각 컬럼의 값을 제곱한 후 합산
            squared_sum = (selected_columns ** 2).sum(axis=1)

            # 합산된 값의 루트를 계산
            root_result = np.sqrt(squared_sum)

            # 새로운 컬럼으로 데이터프레임에 추가
            df[new_column_name] = root_result

            # 기존의 선택된 컬럼들 삭제
            columns_to_drop.extend(selected_columns.columns)
            df.drop(columns=columns_to_drop, inplace=True)
    return df

# 적용할 패턴과 새로운 컬럼 이름의 딕셔너리 생성
patterns = {
    'CURE END POSITION . Collect Result_Dam$': 'CURE END POSITION XYZ Collect Result_Dam',
    'CURE START POSITION . Collect Result_Dam$': 'CURE START POSITION XYZ Collect Result_Dam',
    'HEAD NORMAL COORDINATE . AXIS\(Stage1\) Collect Result_Dam$': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE . AXIS\(Stage2\) Collect Result_Dam$': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE . AXIS\(Stage3\) Collect Result_Dam$': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Dam',
    'HEAD Standby Position . Collect Result_Dam$': 'HEAD Standby Position XYZ Collect Result_Dam',
    'Head Clean Position . Collect Result_Dam$': 'Head Clean Position XYZ Collect Result_Dam',
    'Head Purge Position . Collect Result_Dam$': 'Head Purge Position XYZ Collect Result_Dam',
    'Head Zero Position . Collect Result_Dam$': 'Head Zero Position XYZ Collect Result_Dam',
    'HEAD NORMAL COORDINATE . AXIS\(Stage1\) Collect Result_Fill1': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE . AXIS\(Stage2\) Collect Result_Fill1': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE . AXIS\(Stage3\) Collect Result_Fill1': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Fill1',
    'HEAD Standby Position . Collect Result_Fill1': 'HEAD Standby Position XYZ Collect Result_Fill1',
    'Head Clean Position . Collect Result_Fill1': 'Head Clean Position XYZ Collect Result_Fill1',
    'Head Purge Position . Collect Result_Fill1': 'Head Purge Position XYZ Collect Result_Fill1',
    'CURE END POSITION . Collect Result_Fill2': 'CURE END POSITION XYZ Collect Result_Fill2',
    'CURE STANDBY POSITION . Collect Result_Fill2': 'CURE STANDBY POSITION XYZ Collect Result_Fill2',
    'CURE START POSITION . Collect Result_Fill2': 'CURE START POSITION XYZ Collect Result_Fill2',
    'HEAD NORMAL COORDINATE . AXIS\(Stage1\) Collect Result_Fill2': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage1) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE . AXIS\(Stage2\) Collect Result_Fill2': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage2) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE . AXIS\(Stage3\) Collect Result_Fill2': 'HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Fill2',
    'HEAD Standby Position . Collect Result_Fill2': 'HEAD Standby Position XYZ Collect Result_Fill2',
    'Head Clean Position . Collect Result_Fill2': 'Head Clean Position XYZ Collect Result_Fill2',
    'Head Purge Position . Collect Result_Fill2': 'Head Purge Position XYZ Collect Result_Fill2',
    'Stage1 Circle. Distance Speed Collect Result_Dam': 'Stage1 Circle Distance Speed Collect Result_Dam',
    'Stage1 Line. Distance Speed Collect Result_Dam':'Stage1 Line Distance Speed Collect Result_Dam',
    'Stage2 Circle. Distance Speed Collect Result_Dam': 'Stage2 Circle Distance Speed Collect Result_Dam',
    'Stage2 Line. Distance Speed Collect Result_Dam':'Stage2 Line Distance Speed Collect Result_Dam',
    'Stage3 Circle. Distance Speed Collect Result_Dam': 'Stage3 Circle Distance Speed Collect Result_Dam',
    'Stage3 Line. Distance Speed Collect Result_Dam':'Stage3 Line Distance Speed Collect Result_Dam',
    
}

train_data = add_vector_magnitude(train_data, patterns)
test_data = add_vector_magnitude(test_data, patterns)
train_data

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Fill2,HEAD Standby Position XYZ Collect Result_Fill2,Head Clean Position XYZ Collect Result_Fill2,Head Purge Position XYZ Collect Result_Fill2,Stage1 Circle Distance Speed Collect Result_Dam,Stage1 Line Distance Speed Collect Result_Dam,Stage2 Circle Distance Speed Collect Result_Dam,Stage2 Line Distance Speed Collect Result_Dam,Stage3 Circle Distance Speed Collect Result_Dam,Stage3 Line Distance Speed Collect Result_Dam
0,Dam dispenser #1,AJX75334505,4F1XA938-1,100,16,14.9,8.4,14.7,1.04,0.58,...,672.496914,367.137154,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.0,11358.256909,11600.0
1,Dam dispenser #1,AJX75334505,3KPM0016-2,70,10,21.3,4.9,21.3,1.49,0.34,...,516.633032,274.772633,158.392677,287.445647,8000.000000,8000.000000,18000.000000,18000.0,8000.000000,8000.0
2,Dam dispenser #2,AJX75334501,4E1X9167-1,85,16,14.7,8.5,14.7,1.61,0.93,...,1436.174756,367.004428,129.464281,289.529342,10045.894734,11600.000000,10858.637115,10600.0,11358.256909,11600.0
3,Dam dispenser #2,AJX75334501,3K1X0057-1,70,10,21.3,8.4,21.3,1.49,0.58,...,1514.738885,274.772633,158.392677,287.445647,8000.000000,8000.000000,10000.000000,10000.0,8000.000000,8000.0
4,Dam dispenser #1,AJX75334501,3HPM0007-1,70,10,9.7,4.9,9.6,0.67,0.34,...,516.633032,274.772633,158.392677,287.445647,18000.000000,18000.000000,18000.000000,18000.0,18000.000000,18000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,70,10,17.0,5.0,17.0,1.19,0.35,...,516.633032,274.772633,158.392677,287.445647,10000.000000,10000.000000,18000.000000,18000.0,10000.000000,10000.0
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,100,16,14.9,8.5,14.7,1.04,0.59,...,1436.174756,367.004428,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.0,11358.256909,11600.0
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,100,16,14.2,8.2,14.3,0.99,0.57,...,672.496914,367.137154,129.464281,289.529342,10392.304894,12000.000000,11258.330249,11000.0,11757.976016,12000.0
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,70,10,9.7,4.9,9.7,0.67,0.34,...,1514.738885,274.772633,158.392677,287.445647,18000.000000,18000.000000,18000.000000,18000.0,18000.000000,18000.0


In [5]:
## 중복된 column 제거 ##
train_data = train_data.loc[:, ~train_data.T.duplicated()]

## speed 데이터가 밀린 것으로 보이는 column 제거 ##
train_data = train_data.drop(columns=['THICKNESS 1 Collect Result_Dam'])

In [6]:
# 데이터프레임에서 문자열 데이터가 있는 열에 대해 Label Encoding 적용
label_encoders = {}

for column in train_data.columns:
    if train_data[column].dtype == 'object':  # 문자열 데이터를 확인
        le = LabelEncoder()
        train_data[column] = le.fit_transform(train_data[column].astype(str))  # Label Encoding 적용
        label_encoders[column] = le  # 나중에 디코딩할 수 있도록 레이블 인코더 저장
        
for column in test_data.columns:
    if test_data[column].dtype == 'object':  # 문자열 데이터를 확인
        le = LabelEncoder()
        test_data[column] = le.fit_transform(test_data[column].astype(str))  # Label Encoding 적용
        label_encoders[column] = le  # 나중에 디코딩할 수 있도록 레이블 인코더 저장

        
train_data = train_data.astype(float)
test_data = test_data.astype(float)

train_data

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Fill2,HEAD Standby Position XYZ Collect Result_Fill2,Head Clean Position XYZ Collect Result_Fill2,Head Purge Position XYZ Collect Result_Fill2,Stage1 Circle Distance Speed Collect Result_Dam,Stage1 Line Distance Speed Collect Result_Dam,Stage2 Circle Distance Speed Collect Result_Dam,Stage2 Line Distance Speed Collect Result_Dam,Stage3 Circle Distance Speed Collect Result_Dam,Stage3 Line Distance Speed Collect Result_Dam
0,0.0,3.0,657.0,100.0,16.0,14.9,8.4,14.7,1.04,0.58,...,672.496914,367.137154,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.0,11358.256909,11600.0
1,0.0,3.0,283.0,70.0,10.0,21.3,4.9,21.3,1.49,0.34,...,516.633032,274.772633,158.392677,287.445647,8000.000000,8000.000000,18000.000000,18000.0,8000.000000,8000.0
2,1.0,0.0,589.0,85.0,16.0,14.7,8.5,14.7,1.61,0.93,...,1436.174756,367.004428,129.464281,289.529342,10045.894734,11600.000000,10858.637115,10600.0,11358.256909,11600.0
3,1.0,0.0,251.0,70.0,10.0,21.3,8.4,21.3,1.49,0.58,...,1514.738885,274.772633,158.392677,287.445647,8000.000000,8000.000000,10000.000000,10000.0,8000.000000,8000.0
4,0.0,0.0,142.0,70.0,10.0,9.7,4.9,9.6,0.67,0.34,...,516.633032,274.772633,158.392677,287.445647,18000.000000,18000.000000,18000.000000,18000.0,18000.000000,18000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0.0,0.0,238.0,70.0,10.0,17.0,5.0,17.0,1.19,0.35,...,516.633032,274.772633,158.392677,287.445647,10000.000000,10000.000000,18000.000000,18000.0,10000.000000,10000.0
40502,1.0,0.0,643.0,100.0,16.0,14.9,8.5,14.7,1.04,0.59,...,1436.174756,367.004428,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.0,11358.256909,11600.0
40503,0.0,0.0,540.0,100.0,16.0,14.2,8.2,14.3,0.99,0.57,...,672.496914,367.137154,129.464281,289.529342,10392.304894,12000.000000,11258.330249,11000.0,11757.976016,12000.0
40504,1.0,0.0,164.0,70.0,10.0,9.7,4.9,9.7,0.67,0.34,...,1514.738885,274.772633,158.392677,287.445647,18000.000000,18000.000000,18000.000000,18000.0,18000.000000,18000.0


In [7]:
## Equipment 통합하여 파생변수 생성 - 셋중 하나가 다를 경우, 무조건 Abnormal 이나옴, 근데 34개의 case 밖에 안되서 일반화 시키기 좀 그럴지도##
# train_data['Equipment_Dam'] = pd.to_numeric(train_data['Equipment_Dam'], errors='coerce')
# train_data['Equipment_Fill1'] = pd.to_numeric(train_data['Equipment_Fill1'], errors='coerce')
# train_data['Equipment_Fill2'] = pd.to_numeric(train_data['Equipment_Fill2'], errors='coerce')

train_data['combined'] = train_data[['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2']].sum(axis=1)
train_data['Equipment'] = train_data['combined'].apply(lambda x: 1 if 1 <= x < 3 else 0)
train_data = train_data.drop(columns=['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2', 'combined'])

test_data['combined'] = test_data[['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2']].sum(axis=1)
test_data['Equipment'] = test_data['combined'].apply(lambda x: 1 if 1 <= x < 3 else 0)
test_data = test_data.drop(columns=['Equipment_Dam', 'Equipment_Fill1', 'Equipment_Fill2', 'combined'])

train_data

,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,...,HEAD Standby Position XYZ Collect Result_Fill2,Head Clean Position XYZ Collect Result_Fill2,Head Purge Position XYZ Collect Result_Fill2,Stage1 Circle Distance Speed Collect Result_Dam,Stage1 Line Distance Speed Collect Result_Dam,Stage2 Circle Distance Speed Collect Result_Dam,Stage2 Line Distance Speed Collect Result_Dam,Stage3 Circle Distance Speed Collect Result_Dam,Stage3 Line Distance Speed Collect Result_Dam,Equipment
0,3.0,657.0,100.0,16.0,14.9,8.4,14.7,1.04,0.58,1.02,...,367.137154,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.0,11358.256909,11600.0,0
1,3.0,283.0,70.0,10.0,21.3,4.9,21.3,1.49,0.34,1.49,...,274.772633,158.392677,287.445647,8000.000000,8000.000000,18000.000000,18000.0,8000.000000,8000.0,0
2,0.0,589.0,85.0,16.0,14.7,8.5,14.7,1.61,0.93,1.61,...,367.004428,129.464281,289.529342,10045.894734,11600.000000,10858.637115,10600.0,11358.256909,11600.0,0
3,0.0,251.0,70.0,10.0,21.3,8.4,21.3,1.49,0.58,1.49,...,274.772633,158.392677,287.445647,8000.000000,8000.000000,10000.000000,10000.0,8000.000000,8000.0,0
4,0.0,142.0,70.0,10.0,9.7,4.9,9.6,0.67,0.34,0.67,...,274.772633,158.392677,287.445647,18000.000000,18000.000000,18000.000000,18000.0,18000.000000,18000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0.0,238.0,70.0,10.0,17.0,5.0,17.0,1.19,0.35,1.19,...,274.772633,158.392677,287.445647,10000.000000,10000.000000,18000.000000,18000.0,10000.000000,10000.0,0
40502,0.0,643.0,100.0,16.0,14.9,8.5,14.7,1.04,0.59,1.02,...,367.004428,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.0,11358.256909,11600.0,0
40503,0.0,540.0,100.0,16.0,14.2,8.2,14.3,0.99,0.57,1.00,...,367.137154,129.464281,289.529342,10392.304894,12000.000000,11258.330249,11000.0,11757.976016,12000.0,0
40504,0.0,164.0,70.0,10.0,9.7,4.9,9.7,0.67,0.34,0.67,...,274.772633,158.392677,287.445647,18000.000000,18000.000000,18000.000000,18000.0,18000.000000,18000.0,0


In [8]:
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.metrics import classification_report

X = train_data.drop("target", axis = 1)
y = train_data["target"]



# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

enn = EditedNearestNeighbours(n_neighbors=5)
#X_data, y_data = enn.fit_resample(X_train, y_train)
X_data, y_data = enn.fit_resample(X, y)
X_data

,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,...,HEAD Standby Position XYZ Collect Result_Fill2,Head Clean Position XYZ Collect Result_Fill2,Head Purge Position XYZ Collect Result_Fill2,Stage1 Circle Distance Speed Collect Result_Dam,Stage1 Line Distance Speed Collect Result_Dam,Stage2 Circle Distance Speed Collect Result_Dam,Stage2 Line Distance Speed Collect Result_Dam,Stage3 Circle Distance Speed Collect Result_Dam,Stage3 Line Distance Speed Collect Result_Dam,Equipment
0,1.0,481.0,70.0,16.0,14.2,8.1,14.3,0.99,0.56,1.00,...,367.137154,129.464281,289.529342,10392.306577,12000.000000,11258.330249,11000.000000,11757.976016,12000.0,0
1,0.0,239.0,70.0,10.0,17.0,4.9,17.0,1.19,0.34,1.19,...,274.772633,158.392677,287.445647,10000.000000,10000.000000,18000.000000,18000.000000,10000.000000,10000.0,0
2,3.0,598.0,85.0,16.0,14.9,8.4,14.7,1.63,0.92,1.61,...,367.137154,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.000000,11358.256909,11600.0,0
3,0.0,392.0,70.0,16.0,13.2,7.5,13.2,0.92,0.52,0.92,...,367.137154,129.464281,289.529342,11258.330294,13000.000000,13000.000000,12041.594579,13000.000000,13000.0,0
4,0.0,276.0,70.0,10.0,21.3,10.6,21.3,1.49,0.74,1.49,...,367.137154,129.464281,289.529342,6928.203302,8000.000000,8000.000000,8000.000000,8000.000000,8000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32163,0.0,591.0,85.0,16.0,14.7,8.4,14.7,1.61,0.92,1.61,...,367.137154,129.464281,289.529342,10045.894734,11600.000000,10858.637115,10600.000000,11358.256909,11600.0,0
32164,0.0,306.0,70.0,10.0,21.3,10.5,21.3,1.49,0.73,1.49,...,367.004428,129.464281,289.529342,6928.203302,8000.000000,8000.000000,8000.000000,8000.000000,8000.0,0
32165,0.0,238.0,70.0,10.0,17.0,5.0,17.0,1.19,0.35,1.19,...,274.772633,158.392677,287.445647,10000.000000,10000.000000,18000.000000,18000.000000,10000.000000,10000.0,0
32166,0.0,643.0,100.0,16.0,14.9,8.5,14.7,1.04,0.59,1.02,...,367.004428,129.464281,289.529342,10045.894734,11501.304274,10858.637115,10600.000000,11358.256909,11600.0,0


In [9]:
randomforest_model = RandomForestClassifier(random_state=42) # class_weight 적용 가능
randomforest_model.fit(X_data, y_data)
sfm = SelectFromModel(randomforest_model, threshold='1.25*median')
selected_features = X_data.columns[sfm.get_support()]
X_data = X_data[selected_features]
selected_features

Index(['Model.Suffix_Dam', 'Workorder_Dam',
       'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Machine Tact time Collect Result_Dam', 'PalletID Collect Result_Dam',
       'Production Qty Collect Result_Dam', 'Receip No Collect Result_Dam',
       '1st Pressure Collect Result_AutoClave',
       '2nd Pressure Collect Result_AutoClave',
       '3rd Pressure Collect Result_AutoClave',
       'Chamber Temp. Collect Result_AutoClave',
       'Chamber Temp. Unit Time_AutoClave',
       'Chamber Temp. Judge Value_AutoClave',
       'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
       'Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) C

In [10]:
from imblearn.over_sampling import RandomOverSampler

# 랜덤 오버샘플링 적용
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_data, y_data)

normal = []
abnormal = []
for i in y_resampled:
    if i ==1:
        normal.append(i)
    else:
        abnormal.append(i)

print(len(normal) + len(abnormal))
print(len(normal))
print(len(abnormal))

59636
29818
29818


In [12]:
over_X = pd.concat([X_resampled, y_resampled], axis=1)

# Model.Suffix_Dam과 target으로 그룹화하고 개수를 계산합니다.
result_suffix = over_X.groupby(['Model.Suffix_Dam', 'target']).size().unstack(fill_value=0)

# 비율 계산
result_suffix_percentage = result_suffix.div(result_suffix.sum(axis=1), axis=0) * 100

print("Model.Suffix_Dam별 target 분포 (개수):")
print(result_suffix)
print("\nModel.Suffix_Dam별 target 분포 (비율 %):")
print(result_suffix_percentage)

# Model.Suffix_Dam의 고유값 확인
unique_suffixes = train_data['Model.Suffix_Dam'].unique()
print("\nModel.Suffix_Dam의 고유값:")
print(unique_suffixes)

Model.Suffix_Dam별 target 분포 (개수):
target              0.0    1.0
Model.Suffix_Dam              
0.0               24323  24890
1.0                2976   2440
2.0                 535    107
3.0                1637   1974
4.0                 102     94
5.0                 214    266
6.0                  31     47

Model.Suffix_Dam별 target 분포 (비율 %):
target                  0.0        1.0
Model.Suffix_Dam                      
0.0               49.423933  50.576067
1.0               54.948301  45.051699
2.0               83.333333  16.666667
3.0               45.333703  54.666297
4.0               52.040816  47.959184
5.0               44.583333  55.416667
6.0               39.743590  60.256410

Model.Suffix_Dam의 고유값:
[3. 0. 1. 5. 4. 6. 2.]


In [103]:
catboost_model = CatBoostClassifier(random_seed=42, verbose=False)
xgboost_model = XGBClassifier(random_seed = 42)
randomforest_model = RandomForestClassifier(random_state=42) # class_weight 적용 가능

In [104]:
voting_model = VotingClassifier(estimators=[
    ('catboost', catboost_model),
    ('xgboost', xgboost_model),
    ('randomforest', randomforest_model)
], voting='soft')# 'soft'는 각 모델의 예측 확률 평균을 사용, 'hard'는 각 모델의 최종 예측 결과의 다수결을 사용

In [105]:
S_kfold = StratifiedKFold(n_splits=5)

n_iter = 0

for train_index, test_index in S_kfold.split(X_resampled, y_resampled):  
    x_train_k_fold, x_test_k_fold = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train_k_fold, y_test_k_fold = y_resampled.iloc[train_index], y_resampled.iloc[test_index]
    
    voting_model.fit(x_train_k_fold, y_train_k_fold)
    pred = voting_model.predict(x_test_k_fold)
    n_iter += 1
    
    print(classification_report(y_test_k_fold, pred))

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:23:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      5964
         1.0       1.00      0.95      0.97      5964

    accuracy                           0.97     11928
   macro avg       0.97      0.97      0.97     11928
weighted avg       0.97      0.97      0.97     11928



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:23:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      5964
         1.0       1.00      0.96      0.98      5963

    accuracy                           0.98     11927
   macro avg       0.98      0.98      0.98     11927
weighted avg       0.98      0.98      0.98     11927



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:24:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      5964
         1.0       1.00      0.95      0.97      5963

    accuracy                           0.97     11927
   macro avg       0.97      0.97      0.97     11927
weighted avg       0.97      0.97      0.97     11927



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:24:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      5963
         1.0       1.00      0.96      0.98      5964

    accuracy                           0.98     11927
   macro avg       0.98      0.98      0.98     11927
weighted avg       0.98      0.98      0.98     11927



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:24:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      5963
         1.0       1.00      0.96      0.98      5964

    accuracy                           0.98     11927
   macro avg       0.98      0.98      0.98     11927
weighted avg       0.98      0.98      0.98     11927



In [106]:
test_data = test_data[X_resampled.columns]
test_data

,Model.Suffix_Dam,Workorder_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Machine Tact time Collect Result_Dam,PalletID Collect Result_Dam,Production Qty Collect Result_Dam,Receip No Collect Result_Dam,...,Receip No Collect Result_Fill1,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,HEAD NORMAL COORDINATE XYZ AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE XYZ AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE XYZ AXIS(Stage3) Collect Result_Fill1,Equipment
0,0.0,240.0,17.0,4.9,17.0,0.34,58.00,13.0,195.0,1.0,...,1.0,19.8,13.0,195.0,1.0,1382.313713,1414.123051,1420.980711,1354.301049,0
1,0.0,509.0,14.2,8.3,14.2,0.58,265.02,60.5,14.0,256.0,...,256.0,85.0,19.8,14.0,256.0,1311.119613,1485.584737,1585.405738,1420.854614,0
2,0.0,128.0,9.7,4.9,9.7,0.34,88.30,1.0,98.0,1.0,...,1.0,19.7,1.0,98.0,1.0,661.434169,499.102605,675.057849,519.491367,0
3,0.0,306.0,21.3,10.6,21.3,0.74,265.00,73.2,14.0,0.0,...,0.0,85.0,20.0,14.0,0.0,1313.029132,1382.920649,1585.268441,1420.980711,0
4,0.0,415.0,13.2,7.5,13.2,0.52,265.02,54.3,1.0,215.0,...,215.0,85.0,19.8,1.0,215.0,724.298097,660.536080,973.773362,674.679309,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0.0,277.0,21.3,10.6,21.3,0.74,265.00,73.0,14.0,131.0,...,131.0,85.0,19.5,14.0,131.0,1312.805249,1382.381048,1585.268441,1420.980711,0
17357,0.0,439.0,13.2,7.6,13.2,0.53,265.02,57.7,12.0,279.0,...,279.0,85.0,19.8,12.0,279.0,1312.573034,1486.928761,1585.141541,1420.693417,0
17358,0.0,314.0,13.2,6.6,13.2,0.72,265.00,53.8,4.0,66.0,...,66.0,85.0,20.5,4.0,66.0,725.576836,661.934396,973.773362,674.611370,0
17359,0.0,8.0,9.7,3.9,9.7,0.27,92.20,1.0,117.0,1.0,...,1.0,18.9,1.0,117.0,1.0,661.520249,499.377152,675.731263,519.441036,0


In [107]:
test_pred = voting_model.predict(test_data)
test_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [108]:
result = ['Normal' if pred == 1 else 'AbNormal' for pred in test_pred]
result

['Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',

In [109]:
nor = []
ab = []
for i in result:
    if i == 'Normal':
        nor.append(i)
    else:
        ab.append(i)
print(len(nor))
print(len(ab))

15407
1954


In [110]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = result

df_sub

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Normal
17357,ffed8923c8a448a98afc641b770be153,Normal
17358,fff1e73734da40adbe805359b3efb462,Normal
17359,fff8e38bdd09470baf95f71e92075dec,Normal


In [111]:
df_sub.to_csv("submission.csv", index=False)